# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [2]:
from sklearn.datasets import  fetch_california_housing
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [3]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [4]:
spaceship.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [5]:
spaceship.dropna(inplace=True)
spaceship.drop(columns=['PassengerId','Name'], inplace=True)
spaceship['Cabin'] = spaceship['Cabin'].str[:1]


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [6]:
#your code here

dummy_dict = {}

for column in spaceship.columns:
    if spaceship[column].dtype == 'object':
        dummies = pd.get_dummies(spaceship[column], drop_first=True, prefix=f'is_{column}')
        dummy_dict[column] = dummies
        spaceship.drop(columns=column, inplace=True)
        
spaceship_dummy = pd.concat(dummy_dict.values(), axis=1)

spaceship_with_dummies = pd.concat([spaceship, spaceship_dummy], axis=1)

train = spaceship_with_dummies.drop(columns=['Transported'])

target = spaceship_with_dummies['Transported']

In [7]:
train.dtypes

Age                             float64
RoomService                     float64
FoodCourt                       float64
ShoppingMall                    float64
Spa                             float64
VRDeck                          float64
is_HomePlanet_Europa              uint8
is_HomePlanet_Mars                uint8
is_CryoSleep_True                 uint8
is_Cabin_B                        uint8
is_Cabin_C                        uint8
is_Cabin_D                        uint8
is_Cabin_E                        uint8
is_Cabin_F                        uint8
is_Cabin_G                        uint8
is_Cabin_T                        uint8
is_Destination_PSO J318.5-22      uint8
is_Destination_TRAPPIST-1e        uint8
is_VIP_True                       uint8
dtype: object

In [8]:
##Split

X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.20, random_state=0)

In [9]:
X_train

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,is_HomePlanet_Europa,is_HomePlanet_Mars,is_CryoSleep_True,is_Cabin_B,is_Cabin_C,is_Cabin_D,is_Cabin_E,is_Cabin_F,is_Cabin_G,is_Cabin_T,is_Destination_PSO J318.5-22,is_Destination_TRAPPIST-1e,is_VIP_True
3432,32.0,0.0,0.0,0.0,0.0,0.0,0,1,1,0,0,0,1,0,0,0,0,1,0
7312,4.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,1,0,0,1,0
2042,30.0,0.0,236.0,0.0,1149.0,0.0,0,0,0,0,0,0,0,1,0,0,0,1,0
4999,17.0,13.0,0.0,565.0,367.0,1.0,0,1,0,0,0,0,1,0,0,0,0,1,0
5755,26.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6518,53.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,1,0,0,0,0,1,0
4317,36.0,0.0,0.0,0.0,725.0,2.0,0,0,0,0,0,0,0,1,0,0,0,0,0
2214,36.0,0.0,4756.0,0.0,7818.0,96.0,1,0,0,0,0,1,0,0,0,0,0,1,0
3468,34.0,0.0,4.0,0.0,685.0,1779.0,1,0,0,0,0,1,0,0,0,0,0,1,1


In [10]:
y_train

3432    False
7312     True
2042    False
4999    False
5755     True
        ...  
6518    False
4317    False
2214    False
3468    False
3642    False
Name: Transported, Length: 5284, dtype: bool

In [11]:
#NORMALISATION

from sklearn.preprocessing import StandardScaler, MinMaxScaler

##CONTINOUS

# Separate continuous and dummy features

#####for X_train

X_train_continous = X_train.select_dtypes(include=['float64', 'int64'])
X_train_categorical = X_train.select_dtypes(include=['uint8'])

#####for X_test

X_test_continous = X_test.select_dtypes(include=['float64', 'int64'])
X_test_categorical = X_test.select_dtypes(include=['uint8'])

# Get the column names
column_names_continuous = X_train_continous.columns

# Initialize scaler
scaler = MinMaxScaler()

#Fit

scaler.fit(X_train_continous)

# Scale continuous variables and create a df
X_train_continous_scaled = scaler.transform(X_train_continous)
X_train_continuous_df = pd.DataFrame(X_train_continous_scaled, columns=column_names_continuous)

X_test_continous_scaled = scaler.transform(X_test_continous)
X_test_continuous_df = pd.DataFrame(X_test_continous_scaled, columns=column_names_continuous)

## join the df
X_train_norm = pd.DataFrame(np.concatenate((X_train_continuous_df, X_train_categorical),axis=1))
X_test_norm = pd.DataFrame(np.concatenate((X_test_continuous_df, X_test_categorical),axis=1))

##note that when concatenating pandas convert the uint8 to float64

##we get a lot of new nulls, let do imputation (replace them by the mean)

X_train_norm.dropna(inplace=True)
X_test_norm.dropna(inplace=True)

In [12]:
##this does not work
# X_train_df2 = pd.concat([X_train_continuous_df, X_train_categorical], axis=1)
# X_train_df2

**Perform Train Test Split**

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [13]:
#your code here
#Create a bagging regressor with the following parameters: 

#Base Estimator: DecisionTreeRegressor with a maximum depth of 20.
#"20 splits or levels"
# Number of Estimators (base models): 100.
#number of estimators refers to the total count of base models (decision trees) in the ensemble.
# Maximum Samples per Estimator: 1000.
#randomly selecting a subset of the training data (with replacement), each decision tree is trained on a different subset.

bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)

#Training Bagging model with our normalized data

bagging_reg.fit(X_train_norm, y_train)

BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                 max_samples=1000, n_estimators=100)

In [14]:
pred = bagging_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", bagging_reg.score(X_test_norm, y_test))

MAE 0.27804104028855314
RMSE 0.3782611519333941
R2 score 0.427674003752087


/Users/martinoalonso/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [15]:
#PASTING
pasting_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000,
                               bootstrap=False)

# Train the pasting model
pasting_reg.fit(X_train_norm, y_train)

BaggingRegressor(bootstrap=False, estimator=DecisionTreeRegressor(max_depth=20),
                 max_samples=1000, n_estimators=100)

- Random Forests

In [16]:
pred_bagging = bagging_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred_bagging, y_test))
print("RMSE", mean_squared_error(pred_bagging, y_test, squared=False))
print("R2 score", bagging_reg.score(X_test_norm, y_test))


MAE 0.27804104028855314
RMSE 0.3782611519333941
R2 score 0.427674003752087


/Users/martinoalonso/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Gradient Boosting

In [ ]:
#your code here
gb_reg = GradientBoostingRegressor(max_depth=20,
                                   n_estimators=100)

gb_reg.fit(X_train_norm, y_train)

pred_gb = gb_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred_gb, y_test))
print("RMSE", mean_squared_error(pred_gb, y_test, squared=False))
print("R2 score", gb_reg.score(X_test_norm, y_test))


MAE 0.260643115916718
RMSE 0.41224207989192
R2 score 0.3202258702655354


/Users/martinoalonso/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Adaptive Boosting

In [ ]:
#your code here
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20),
                            n_estimators=100)

ada_reg.fit(X_train_norm, y_train)

pred_ada = ada_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred_ada, y_test))
print("RMSE", mean_squared_error(pred_ada, y_test, squared=False))
print("R2 score", ada_reg.score(X_test_norm, y_test))

MAE 0.24653826061989856
RMSE 0.42037328147474085
R2 score 0.29314521688863315


/Users/martinoalonso/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Which model is the best and why?

In [ ]:
#comment here
#bagging

In [ ]:
## FOREST
forest = RandomForestRegressor(n_estimators=100,
                             max_depth=20)

forest.fit(X_train_norm, y_train)


RandomForestRegressor(max_depth=20)

In [ ]:
pred_forest = forest.predict(X_test_norm)

print("MAE", mean_absolute_error(pred_forest, y_test))
print("RMSE", mean_squared_error(pred_forest, y_test, squared=False))
print("R2 score", forest.score(X_test_norm, y_test))

MAE 0.2689431158517582
RMSE 0.38336232085980876
R2 score 0.4121333237799242


/Users/martinoalonso/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
